In [1]:
import os
from bs4 import BeautifulSoup
import chardet
import csv
import re
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from time import sleep
import csv

In [2]:
chrome_options = webdriver.ChromeOptions()
ua = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.102 Safari/537.36 Edg/104.0.1293.63'
chrome_options.add_argument('user-agent=%s'%ua)
# chrome_options.add_argument("window-size=1200x600")
chrome_options.add_argument('--headless')# now show the browser

In [3]:
# get all the jobs located in different cities
# driver = webdriver.Chrome('C:/Program Files (x86)/Google/Chrome/Application/chromedriver.exe', options=chrome_options)
# wait = WebDriverWait(driver, 5)

# this is a glassdoor crawler
# html=['https://www.ziprecruiter.com/candidate/search?form=jobs-landing&search=&location=CA']
# la_page_base = 'https://www.glassdoor.com/Job/los-angeles-jobs-SRCH_IL.0,11_IC1146821.htm'
# sf_page_base = 'https://www.glassdoor.com/Job/san-francisco-jobs-SRCH_IL.0,13_IC1147401.htm'
# seattle_page_base = 'https://www.glassdoor.com/Job/seattle-seattle-jobs-SRCH_IL.0,7_IC1150505_KO8,15.htm'
# nj_base_page = 'https://www.glassdoor.com/Job/new-jersey-jobs-SRCH_IL.0,10_IS39.htm'
# texas_base_page = 'https://www.glassdoor.com/Job/texas-jobs-SRCH_IL.0,5_IS1347.htm'
# florida_base_page = 'https://www.glassdoor.com/Job/florida-jobs-SRCH_IL.0,7_IS3318.htm'
# ny_base_page = 'https://www.glassdoor.com/Job/new-york-state-jobs-SRCH_IL.0,14_IS428.htm'
# ma_base_page = 'https://www.glassdoor.com/Job/massachusetts-jobs-SRCH_IL.0,13_IS3399.htm'
# pa_base_page = 'https://www.glassdoor.com/Job/pennsylvania-jobs-SRCH_IL.0,12_IS3185.htm'
# ge_base_page = 'https://www.glassdoor.com/Job/georgia-jobs-SRCH_IL.0,7_IS3426.htm'
# il_base_page = 'https://www.glassdoor.com/Job/illinois-jobs-SRCH_IL.0,8_IS302.htm'
# or_base_page = 'https://www.glassdoor.com/Job/oregon-jobs-SRCH_IL.0,6_IS3163.htm'
# az_base_page = 'https://www.glassdoor.com/Job/arizona-jobs-SRCH_IL.0,7_IS483.htm'

# driver.get(nj_base_page)
base_urls = [
    'https://www.glassdoor.com/Job/data-scientist-jobs-SRCH_KO0,14.htm',
    'https://www.glassdoor.com/Job/hr-jobs-SRCH_KO0,2.htm',
    'https://www.glassdoor.com/Job/lawyer-jobs-SRCH_KO0,6.htm',
    'https://www.glassdoor.com/Job/artist-jobs-SRCH_KO0,6.htm',
    'https://www.glassdoor.com/Job/web-designer-jobs-SRCH_KO0,12.htm',
    'https://www.glassdoor.com/Job/mechanical-engineer-jobs-SRCH_KO0,19.htm',
    'https://www.glassdoor.com/Job/salesman-jobs-SRCH_KO0,8.htm',
    'https://www.glassdoor.com/Job/civil-engineer-jobs-SRCH_KO0,14.htm',
    'https://www.glassdoor.com/Job/software-developer-jobs-SRCH_KO0,18.htm',
    'https://www.glassdoor.com/Job/business-analyst-jobs-SRCH_KO0,16.htm',
    'https://www.glassdoor.com/Job/automation-tester-jobs-SRCH_KO0,17.htm',
    'https://www.glassdoor.com/Job/electrical-engineer-jobs-SRCH_KO0,19.htm',
    'https://www.glassdoor.com/Job/operations-manager-jobs-SRCH_KO0,18.htm',
    'https://www.glassdoor.com/Job/network-security-engineer-jobs-SRCH_KO0,25.htm',
    'https://www.glassdoor.com/Job/product-manager-jobs-SRCH_KO0,15.htm'
]
# len(base_urls)

In [231]:
page_end = driver.find_element_by_xpath('.//div[@data-test="pagination-footer-text"]').text
page_count = int(page_end.split(' ')[-1])
page_count
driver.close()

In [192]:
page_count

30

In [6]:
# get all the urls of jobs in LA
htmls_nj = []
# https://www.glassdoor.com/Job/los-angeles-jobs-SRCH_IL.0,11_IC1146821_IP3.htm
for i in range(1,31):
    page = nj_base_page.rstrip('.htm')
    new_page = page+'_IP'+str(i)+'.htm'
    htmls_nj.append(new_page)
htmls_nj

['https://www.glassdoor.com/Job/new-jersey-jobs-SRCH_IL.0,10_IS39_IP1.htm',
 'https://www.glassdoor.com/Job/new-jersey-jobs-SRCH_IL.0,10_IS39_IP2.htm',
 'https://www.glassdoor.com/Job/new-jersey-jobs-SRCH_IL.0,10_IS39_IP3.htm',
 'https://www.glassdoor.com/Job/new-jersey-jobs-SRCH_IL.0,10_IS39_IP4.htm',
 'https://www.glassdoor.com/Job/new-jersey-jobs-SRCH_IL.0,10_IS39_IP5.htm',
 'https://www.glassdoor.com/Job/new-jersey-jobs-SRCH_IL.0,10_IS39_IP6.htm',
 'https://www.glassdoor.com/Job/new-jersey-jobs-SRCH_IL.0,10_IS39_IP7.htm',
 'https://www.glassdoor.com/Job/new-jersey-jobs-SRCH_IL.0,10_IS39_IP8.htm',
 'https://www.glassdoor.com/Job/new-jersey-jobs-SRCH_IL.0,10_IS39_IP9.htm',
 'https://www.glassdoor.com/Job/new-jersey-jobs-SRCH_IL.0,10_IS39_IP10.htm',
 'https://www.glassdoor.com/Job/new-jersey-jobs-SRCH_IL.0,10_IS39_IP11.htm',
 'https://www.glassdoor.com/Job/new-jersey-jobs-SRCH_IL.0,10_IS39_IP12.htm',
 'https://www.glassdoor.com/Job/new-jersey-jobs-SRCH_IL.0,10_IS39_IP13.htm',
 'https:

In [6]:
job_title_list = []
company_list = []
location_list = []
job_type_list = []
job_description_list = []
salary_list = []
requirement_list = []
apply_url_list = []

In [45]:
# open different pages and download, relatively slow
import time
begin_time = time.time()
count = 0
for html in htmls:
    count += 1
    driver = webdriver.Chrome('C:/Program Files (x86)/Google/Chrome/Application/chromedriver.exe', options=chrome_options)
    wait = WebDriverWait(driver, 2)
    driver.get(html)
    try:
        if driver.find_element_by_xpath('.//div[@class="fullContent"]'):
            close_button = driver.find_element_by_xpath('.//span[@alt="Close"]').click()
            sleep(1)
    except:
        sleep(1) 
#     sleep(1)
    job_list = driver.find_elements_by_xpath('.//ul/li[@data-test="jobListing"]')
    for job in job_list:
        url = job.find_element_by_xpath('.//a[@data-test="job-link"]').get_attribute('href')
        # only record the new urls
        if url not in apply_url_list:
            apply_url_list.append(url)
    end_time = time.time()
    time_gap = end_time-begin_time
    print('%d pages completed. %d urls downloaded. %s time spent.' %(count, len(apply_url_list), str(time_gap)))
    driver.close()

In [9]:
# directly click the next page
import time
begin_time = time.time()
url_list = []
def get_urls(base_url):
    driver = webdriver.Chrome('C:/Program Files (x86)/Google/Chrome/Application/chromedriver.exe', options=chrome_options)
    wait = WebDriverWait(driver, 5)
    driver.get(base_url)
    sleep(1)
    next_page = driver.find_element_by_xpath('.//span[@alt="next-icon"]')
    count = 0
    # url_list = []
    page_end = driver.find_element_by_xpath('.//div[@data-test="pagination-footer-text"]').text
    page_count = int(page_end.split(' ')[-1]) # get the length of all the pages

    while next_page:
        count += 1
        try:
            if driver.find_element_by_xpath('.//div[@class="fullContent"]'):
                close_button = driver.find_element_by_xpath('.//span[@alt="Close"]').click()
                sleep(1)
        except:
            sleep(1) 
        sleep(1)
        job_list = driver.find_elements_by_xpath('.//ul/li[@data-test="jobListing"]')
        for job in job_list:

            url = job.find_element_by_xpath('.//a[@data-test="job-link"]').get_attribute('href')
            # only record the new urls
            if url not in url_list:
                url_list.append(url)
        end_time = time.time()
        time_gap = end_time-begin_time
        print('page %d completed. %d urls downloaded. %s seconds spent.' %(count, len(url_list), str(time_gap)))
    #     driver.close()
        next_page = driver.find_element_by_xpath('.//span[@alt="next-icon"]')
        next_page.click()
        sleep(4)
        if count==10: # exit when all the pages are downloaded
            driver.close()
            break
    print('Done')

In [10]:
# driver = webdriver.Chrome('C:/Program Files (x86)/Google/Chrome/Application/chromedriver.exe', options=chrome_options)
# wait = WebDriverWait(driver, 2)

for base_url in base_urls:
    get_urls(base_url)
    print('Done')
#     job_title = driver.find_element_by_xpath('.//div[@data-test="job-title"]').text
#     print(job_title)

page 1 completed. 30 urls downloaded. 26.58282732963562 seconds spent.
page 2 completed. 60 urls downloaded. 33.42034935951233 seconds spent.
page 3 completed. 90 urls downloaded. 40.1175799369812 seconds spent.
page 4 completed. 120 urls downloaded. 46.82671523094177 seconds spent.
page 5 completed. 150 urls downloaded. 53.548081398010254 seconds spent.
page 6 completed. 180 urls downloaded. 60.269994020462036 seconds spent.
page 7 completed. 210 urls downloaded. 67.05432868003845 seconds spent.
page 8 completed. 240 urls downloaded. 73.86306595802307 seconds spent.
page 9 completed. 270 urls downloaded. 80.67173504829407 seconds spent.
page 10 completed. 300 urls downloaded. 87.48367404937744 seconds spent.
Done
Done
page 1 completed. 330 urls downloaded. 121.54317665100098 seconds spent.
page 2 completed. 360 urls downloaded. 128.37277173995972 seconds spent.
page 3 completed. 390 urls downloaded. 135.08716750144958 seconds spent.
page 4 completed. 420 urls downloaded. 141.816421985

page 2 completed. 3360 urls downloaded. 1030.4670717716217 seconds spent.
page 3 completed. 3390 urls downloaded. 1037.1808960437775 seconds spent.
page 4 completed. 3420 urls downloaded. 1043.952564239502 seconds spent.
page 5 completed. 3450 urls downloaded. 1050.7436668872833 seconds spent.
page 6 completed. 3480 urls downloaded. 1057.6081490516663 seconds spent.
page 7 completed. 3510 urls downloaded. 1064.4532403945923 seconds spent.
page 8 completed. 3540 urls downloaded. 1071.3372576236725 seconds spent.
page 9 completed. 3570 urls downloaded. 1078.1834843158722 seconds spent.
page 10 completed. 3600 urls downloaded. 1084.9754152297974 seconds spent.
Done
Done
page 1 completed. 3630 urls downloaded. 1108.6564819812775 seconds spent.
page 2 completed. 3660 urls downloaded. 1115.479773759842 seconds spent.
page 3 completed. 3690 urls downloaded. 1122.2259321212769 seconds spent.
page 4 completed. 3720 urls downloaded. 1128.994960784912 seconds spent.
page 5 completed. 3750 urls do

In [11]:
categorys = ['Data Scientist','HR','Lawyer','Artist','Web Designer','Mechanical Engineer','Salesman',
                'Civil Engineer','Software Developer','Business Analyst','Automation Tester','Electrical Engineer',
                'Operations Manager','Network Security Engineer','Product Manager']
category_list = []
for c in categorys:
    category_list += [c]*300
len(category_list)

4500

In [12]:
job_url_df = pd.DataFrame({'url':url_list, 'category':category_list})
job_url_df.to_csv('job_url_category.csv',index=False, encoding = 'utf-8-sig')

In [5]:
job_url_df = pd.read_csv('job_url_category.csv')
url_list = job_url_df['url'].tolist()
len(url_list)

4500

In [10]:
def get_details(url):
    driver = webdriver.Chrome('C:/Program Files (x86)/Google/Chrome/Application/chromedriver.exe', options=chrome_options)
    wait = WebDriverWait(driver, 5)
    driver.get(url)
    sleep(0.5)
    try:
        if driver.find_element_by_xpath('.//div[@data-test="job-title"]'):
            job_title = driver.find_element_by_xpath('.//div[@data-test="job-title"]').text
    except:
        job_title = ''
    try:
        if driver.find_element_by_xpath('.//div[@data-test="employer-name"]'):
            company_rating = driver.find_element_by_xpath('.//div[@data-test="employer-name"]').text
            if len(company_rating.split('\n'))>1:
                company_name = company_rating.split('\n')[0]
                company_score = company_rating.split('\n')[1]
            else:
                company_name = company_rating
                company_score = ''
    except:
        company_name = ''
        company_score = ''
    try:
        if driver.find_element_by_xpath('.//div[@class="css-1rzz8ht ecgq1xb2"]'):
            show_buttom = driver.find_element_by_xpath('.//div[@class="css-1rzz8ht ecgq1xb2"]').click()
#             sleep(1)
    except:
        sleep(0.5)
    try:
        if driver.find_element_by_xpath('.//div[@id="JobDescriptionContainer"]'):
            job_details = driver.find_element_by_xpath('.//div[@id="JobDescriptionContainer"]').text
    except:
        job_details = ''
    try:
        if driver.find_element_by_xpath('.//div[@class="css-1v5elnn e11nt52q2"][1]'):
            location = driver.find_element_by_xpath('.//div[@class="css-1v5elnn e11nt52q2"][1]').text
    except:
        location = ''
    try:
        if driver.find_element_by_xpath('.//div[@class="css-1v5elnn e11nt52q2"][2]/span'):
            salary = driver.find_element_by_xpath('.//div[@class="css-1v5elnn e11nt52q2"][2]/span').text.lstrip('Employer est.:')
    except:
        salary = ''
    
    try:
        if driver.find_element_by_xpath('.//a[@data-test="employerLogo"]'):
            company_url = driver.find_element_by_xpath('.//a[@data-test="employerLogo"]').get_attribute('href')
    except:
        company_url = ''
        
    driver.close()
    data = [job_title,company_name,company_score,job_details,salary,location,url,company_url]
    return data

In [17]:
# create a file to store the data
f = open('glass_door_1121_3.csv','a', newline = '', encoding = 'utf-8-sig')
writer = csv.writer(f)
header = ['source','job_title','company_name','company_score','job_details','salary','location','url','company_url']
writer.writerow(header)

89

In [19]:
# initialize a data_id set to store the already downloaded records
data_id = set()
count = 2500

In [157]:
22+558

580

In [17]:
url_list[0]

'https://www.glassdoor.com/partner/jobListing.htm?pos=101&ao=1110586&s=58&guid=000001849d18ccfb94f874125be5a755&src=GD_JOB_AD&t=SR&vt=w&cs=1_38ff3e7f&cb=1669082959556&jobListingId=1008158851667&cpc=DFCAFF9DFE7B86C3&jrtk=3-0-1giehhj91k6fu801-1giehhj9ii6hq800-75eb9761767a550b--6NYlbfkN0COkaXPVq4ci0oYoPZVLi-OSs5LLhX_qiImzTXPGEjA3ArG44TF1XBM1S9Xs8xK2kyPMAA7fhaLshp56_mNoVsqUtc2Deyb2KyhVggGaMSrupe5wCZI4NBNghF-pw3t3Cvk-L2O6evORA-3pkPCnQGXryCZj0iAp9ZeK0K7FjhZwEk3GkLX5TsK90a6SZMsPM_Zzdvub8toPEV2RkL6t4ZIP5BVMidoo3Tu4rvQpduqIBU2fwKvPN-7bFZTd8nlmL842alRSsf2lX-iDRdkdA5PlrrzvhmD2bqtAIDllOOG0NqpjhSkbxR4wiV348EQJ52jLWg4SHF2nsfLOY6GI5vfQdA3tAeku9xo4kYfu1ebRZFFhw2fdskARn8vaIKMPw6gRIg7Dj6ZGSg9zPy_zUD5IEbF9WZ4extCuCFlR2DgkrRRvgQpAlNIcvRNVOFMGsUHqWx-8WAXJljZgqEZkIKUnjqf5VKrjjelAk8D68becTmvTvJtcwsIZhPDFR4QGi34V3OpbV1uhdkwZHbsY5ujmrHZdPLiZe_2-pDmjcdwawKtTYPNZDn_YWaVGGpYM0c3HMtIurO0OVI-4J9d69GxNF4JHdZx0_dV-9tdWQrNJNb_0jAURbt8iKUCv5M6bLfk4mLuNM3OVpHJyaw2giG3oXL87gYURuQ%3D'

In [31]:
url_list[40]

'https://www.glassdoor.com/partner/jobListing.htm?pos=211&ao=1110586&s=58&guid=000001849d1915bfa72db39a91a5d43b&src=GD_JOB_AD&t=SR&vt=w&ea=1&cs=1_1e0a454a&cb=1669082978184&jobListingId=1008289362002&jrtk=3-0-1giehi5frm6rr801-1giehi5g9gfqd800-d723b740532d4c25--6NYlbfkN0D6OzZjpD_hbicRkMZwNNvvxSeL23iIfvaC4EytleQ8zDIpz0YQ5KbISa7_Zvw6kCw250jdZkkP84VONfxOPmoKCAN8K2RrR2xxdrmCqGRyD4TqW7ggKvYfpxzweBNMAzNWkJ_zKBPgDBGamUR9o8zczjZHlXT2dOAPSmxTNQiOhrjJV5bkPrpojDymNveq8lDxUE_ChuxFcZpoayDtuG1m6c1FK5TcFF5FmTY0wSBt9tapSLEjRsH0C-S8AiZrtrrgwsspcttCscc8ljyNsAnty5fhIs_VuODc9P--aOPa7u02CBc6vWZctiHDQ2Z-aKjEqMx9fS1lpLybP_AFF6QGCDR-4VHax7HxSoKaZXDoWM6eBhUiCLCkfctcOPEeXl8tC6rT-arsm4xJCWFE2sov1cGtRJGZ15tpBlIehi8mSuS0J6IehmUylW7uE1b5VG_OnfogAHdPwfpIgaGC21kwRgLCAIW1mYkTLS4MCcgyj5Zt7v--Sl1IJtFEbMQHqDtXOmF65KVVMfPpEuKXJRN-v2UFbTDt-es%3D'

In [ ]:
begin_time = time.time()
for url in url_list[2500:]:
    data = get_details(url)
    print(data[0:3]) # print the job title and company name
    data_to_write = ['https://www.glassdoor.com/']+data
    sleep(0.8)
    if data and str(data) not in data_id:
        data_id.add(str(data))
        count += 1
        writer.writerow(data_to_write)
        print('%d job downloaded' %count)
end_time = time.time()
time_gap = end_time-begin_time
print('%d jobs downloaded. %s seconds spent.' %(count,str(time_gap)))

['Junior Software Developer', 'AGI – Aggrowth International', '3.6']
2501 job downloaded
['Software Development Engineer (ElasticPath)', 'The Pokémon Company International', '4.2']
2502 job downloaded
['Developer *Remote Flexibility*', 'FORT WAYNE METALS RESEARCH PRODUCTS, LLC', '4.0']
2503 job downloaded
['Software Developer - Python', 'Trinity Solar Careers', '3.8']
2504 job downloaded
['Junior Software Test Engineer', 'InternU', '3.2']
2505 job downloaded
['Mid-Level Front-End Vue & Angular Developer (100% Remote)', 'Web Advanced', '']
2506 job downloaded
['Back-End Developer (Remote)', 'BankUnited', '3.7']
2507 job downloaded
['Front End Developer (React)', 'Experfy Inc', '4.7']
2508 job downloaded
['', '', '']
2509 job downloaded
['Junior Software Developer', 'The Josef Group Inc.', '']
2510 job downloaded
['Senior Web Developer', 'iVisa.com', '']
2511 job downloaded
['Software Developer', 'Contractor Nation (formerly known as Basement Systems, Inc.)', '3.5']
2512 job downloaded
[

2601 job downloaded
['Jr. C# Software Developer (Permanently Remote)', 'Clarity Ventures', '3.8']
2602 job downloaded
['Sr. Software Developer - Remote', 'Symetra', '4.0']
2603 job downloaded
['Software Developer', 'Miles Technologies', '4.5']
2604 job downloaded
['Software Developer - Telecommute Available', 'ImageTrend Inc', '3.9']
2605 job downloaded
['Junior Software Developer', 'Reli.', '']
2606 job downloaded
['Jr. Front End Developer', 'University of Utah', '4.2']
2607 job downloaded
['NetApp Emerging Talent - Software Engineer (SJC)', 'Net App, Inc.', '4.2']
2608 job downloaded
['Application Developer', 'Zillow', '3.9']
2609 job downloaded
['Software Development Engineer I', 'GM Financial', '3.8']
2610 job downloaded
['Software Engineer - Backend', 'Walmart', '3.3']
2611 job downloaded
['Software Development Engineer', 'Zillow', '3.9']
2612 job downloaded
['Senior Software Developer', 'TherapyNotes.com', '4.5']
2613 job downloaded
['Software Developer', 'Branch', '4.8']
2614 jo

2705 job downloaded
['Remote - Contract Business Systems Analyst', 'F.A. Davis Company', '4.1']
2706 job downloaded
['Business Analyst III', 'Chick-fil-A, Inc.', '3.9']
2707 job downloaded
['Business Analyst, Product', 'Healthmap Solutions', '4.2']
2708 job downloaded
['Senior Business Analyst', 'Resource Data', '4.2']
2709 job downloaded
['Technical Business Intelligence Analyst', 'Memorial Hermann Health System', '3.7']
2710 job downloaded
['Business Analyst I', 'UFCW & Employers Trust LLC', '4.1']
2711 job downloaded
['Sr. Business Analyst, Productivity Solutions', 'Chick-fil-A, Inc.', '3.9']
2712 job downloaded
['Business Analyst - Entry Level', 'CGI Group, Inc.', '3.9']
2713 job downloaded
['QA Testing/Business Analyst', 'Genworth', '3.7']
2714 job downloaded
['Sr. Business Analyst', 'Capital One', '4.2']
2715 job downloaded
['FX Risk Business Analyst - AVP', 'Citi', '3.9']
2716 job downloaded
['IT Business Intermediate Analyst', 'Citi', '3.9']
2717 job downloaded
['Business Analy

2806 job downloaded
['Business Analyst', 'MAPFRE', '3.2']
2807 job downloaded
['IT Business Analyst', 'NTT Global Data Centers Americas, Inc.', '3.8']
2808 job downloaded
['Business Analyst', 'Oil Filter Inc', '3.0']
2809 job downloaded
['Junior Business Analyst', 'Digipath Infotech', '']
2810 job downloaded
['Business Analyst - AppDev', 'Health Services Advisory Group, Inc.', '3.1']
2811 job downloaded
['Business Analyst', 'Magnit', '']
2812 job downloaded
['Quantitative Business Analyst', 'Apple', '4.2']
2813 job downloaded
['Business Analyst (C)', 'Harvard Partners, LLP, Trusted Advisors to IT', '2.0']
2814 job downloaded
['Business Analyst (Remote)', 'Excelraise, LLC', '3.8']
2815 job downloaded
['Business Analyst', 'Virtusa', '3.7']
2816 job downloaded
['Business Analyst', 'e business international inc', '']
2817 job downloaded
['Service Now Business Analyst', 'HSK IT Solutions India Pvt. Ltd.', '5.0']
2818 job downloaded
['Business Analyst (IT)', 'Northern Trust Corp.', '3.9']
28

2907 job downloaded
['IT Business Analyst - Contract', 'CyberCoders', '4.2']
2908 job downloaded
['Business Analyst.........Remote', 'Vaco', '3.8']
2909 job downloaded
['Business Analyst- REMOTE in Florida', 'Capital Staffing Solutions', '4.2']
2910 job downloaded
['Business Analyst, Product', 'Healthmap Solutions', '4.2']
2911 job downloaded
['Remote - Contract Business Systems Analyst', 'F.A. Davis Company', '4.1']
2912 job downloaded
['Senior Technical Business Analyst', 'CPS Energy', '3.7']
2913 job downloaded
['Business Systems Analyst (100% Remote)', 'SYSTEMTEC', '4.4']
2914 job downloaded
['Business Analyst III', 'Chick-fil-A, Inc.', '3.9']
2915 job downloaded
['Business Intelligence Analyst 1 (Remote)', 'USAA', '3.6']
2916 job downloaded
['Business Intelligence Analyst III (Trading Card Game)', 'The Pokémon Company International', '4.2']
2917 job downloaded
['Business Intelligence & Reporting Analyst', 'BankUnited', '3.7']
2918 job downloaded
['QA Testing/Business Analyst', 'Ge

...

3805 job downloaded
['VP of Operations', 'Vitality Bowls', '3.5']
3806 job downloaded
['Solar Operations Manager', '360 TalentAvenue', '4.0']
3807 job downloaded
['Manufacturing Operations Manager', 'MW Industries, Inc.', '3.4']
3808 job downloaded
['Product Manager, Operations Data - Alta Petens - REMOTE', 'Takeda Pharmaceutical', '4.0']
3809 job downloaded
['IT Operations Manager', 'Chick-fil-A, Inc.', '3.9']
3810 job downloaded
['Operations Supervisor (REMOTE)', 'CareCentrix', '3.3']
3811 job downloaded
['Remote Escrow Operations Manager', 'First American Financial Corporation', '3.8']
3812 job downloaded
['Assistant Operations Manager DC', 'Williams-Sonoma-Supply Chain', '3.2']
3813 job downloaded
['Global Operations Vendor Manager', 'Zillow', '3.9']
3814 job downloaded
['Operations Manager', 'California Water Service', '3.7']
3815 job downloaded
['Operations Manager', 'YYwireless', '5.0']
3816 job downloaded
['ESPASat Product Line Business Operations Manager 2', 'Northrop Grumman'

3895 job downloaded
['Operations Manager', 'Lineage Logistics', '3.6']
3896 job downloaded
['Area Operations Manager', 'TJ Maxx', '3.7']
3897 job downloaded
['Terminal Operations Manager', 'Confidential', '']
3898 job downloaded
['Global Operations Vendor Manager', 'Zillow', '3.9']
3899 job downloaded
['Revenue Operations Manager', 'Comcast', '3.9']
3900 job downloaded
['Network Security Platform Engineer (Remote Eligible)', 'Capital One', '4.2']
3901 job downloaded
['Cloud Engineer - Network SME', 'TIS 3 Solutions', '']
3902 job downloaded
['2023 Associate Systems Engineer – Charlottesville, VA', 'Northrop Grumman', '4.0']
3903 job downloaded
['Junior Security Operations Center Engineer - Remote', 'Lumen', '3.4']
3904 job downloaded
['2023 Associate Software Engineer – Sykesville, MD', 'Northrop Grumman', '4.0']
3905 job downloaded
['Cloud Infrastructure Engineer', 'The Pokémon Company International', '4.2']
3906 job downloaded
['Network Security Engineer', 'Jack Henry and Associates,

3999 job downloaded
['DevOps Engineer', 'Educe Group', '5.0']
4000 job downloaded
['Cloud Security Engineer', 'KUBRA', '3.9']
4001 job downloaded
['Intermediate Network Engineer', 'Peraton', '3.6']
4002 job downloaded
['Cloud Network Engineer', 'Brooksource', '4.1']
4003 job downloaded
['Security Network Engineer', 'Robert Half', '3.9']
4004 job downloaded
['Machine Learning Engineer', 'Vaco', '3.8']
4005 job downloaded
['Security Engineer', 'CyberCoders', '4.2']
4006 job downloaded
['QA Network Testing Engineer', 'UNIVERSAL Technologies', '3.4']
4007 job downloaded
['Sr. DevOps Engineer', 'R2C Solutions, INC', '']
4008 job downloaded
['Wireless Network Engineer', 'tektree inc', '4.2']
4009 job downloaded
['Cloud Engineer (Remote)', 'Remote Jobs', '3.3']
4010 job downloaded
['Security Engineer', 'zettalogix.Inc', '']
4011 job downloaded
['Jr Systems Development Engineer, AWS Security Infrastructure', 'Amazon.com Services LLC', '3.8']
4012 job downloaded
['Network Cybersecurity Engineer

4101 job downloaded
['Kubernetes Engineer (Remote)', 'Tandym Group', '3.7']
4102 job downloaded
['Cloud Engineer', 'ARC Group', '5.0']
4103 job downloaded
['Principle Network Engineer', 'Harvard Partners, LLP, Trusted Advisors to IT', '2.0']
4104 job downloaded
['Software Test Engineer (Starlink)', 'SpaceX', '3.9']
4105 job downloaded
['AWS DevOps Engineer', 'Lazard Ltd.', '4.2']
4106 job downloaded
['Network Security Engineer', 'INFOMATICS, INC.', '4.0']
4107 job downloaded
['Network Engineering Engineer', 'Salesforce', '4.4']
4108 job downloaded
['product security engineer- Phoenix OR Remote', 'Starbucks', '3.7']
4109 job downloaded
['Principal Cybersecurity Engineer- Network Security (Remote)', 'Home Depot / THD', '3.8']
4110 job downloaded
['Junior Security Operations Center Engineer - Remote', 'Lumen', '3.4']
4111 job downloaded
['2023 Associate Systems Engineer – Charlottesville, VA', 'Northrop Grumman', '4.0']
4112 job downloaded
['Cloud Infrastructure Engineer', 'The Pokémon Co

4200 job downloaded
['Product Manager - ESG Scores', 'Bloomberg', '4.1']
4201 job downloaded
['ESG Data Product Manager', 'Bloomberg', '4.1']
4202 job downloaded
['Data License Platform Product Manager', 'Bloomberg', '4.1']
4203 job downloaded
['Manager, Technical Product - Consumer Technology', 'Alterra Mountain Company', '3.8']
4204 job downloaded
['BVAL Technical Product Manager', 'Bloomberg', '4.1']
4205 job downloaded
['Manager, Product Management', 'Capital One', '4.2']
4206 job downloaded
['Product Manager', 'Lyten', '4.3']
4207 job downloaded
['Product Manager', 'Media News Group', '2.8']
4208 job downloaded
['Product Owner', 'TherapyNotes.com', '4.5']
4209 job downloaded
['Product Manager', 'Comcast', '3.9']
4210 job downloaded
['Customer Resiliency Card - Product Manager - Manager', 'Capital One', '4.2']
4211 job downloaded
['Sr. Manager, Product Management - Channels Product & Platforms', 'Capital One', '4.2']
4212 job downloaded
['Product Manager', 'Point Digital Finance, I

In [16]:
f.close()
# driver.close()

In [33]:
source_list = ['https://www.ziprecruiter.com/']*len(job_title_list)

In [199]:
f1 = open('job_urls_seattle.txt','w')
for url in url_list:
    f1.write(url+'\n')
f1.close()